In [1]:
import numpy as np
import pandas as pd
%pip install scikit-surprise
from surprise import AlgoBase, PredictionImpossible, Reader, Dataset, accuracy
from surprise.model_selection import train_test_split, cross_validate

You should consider upgrading via the '/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
data_train_raw = pd.read_csv('../data/data-train.csv')

# parse rows and columns
row_str = data_train_raw['Id'].apply(lambda x: x.split('_')[0])
row_id = row_str.apply(lambda x: int(x.split('r')[1]) - 1)
col_str = data_train_raw['Id'].apply(lambda x: x.split('_')[1])
col_id = col_str.apply(lambda x: int(x.split('c')[1]) - 1)

# apply changes
data_train_raw['row'] = row_id
data_train_raw['col'] = col_id

# dataset as data frame
data_train_df = data_train_raw.loc[:,['row', 'col', 'Prediction']]

In [4]:
# set up surprise dataset
reader = Reader()
dataset = Dataset.load_from_df(data_train_df[['row', 'col', 'Prediction']], reader)

# now set up training and test set, with a test split of 25%
trainset, testset = train_test_split(dataset, test_size=0.25)

In [5]:
type(trainset)

surprise.trainset.Trainset

In [ ]:
def als_objective():
    regularization = lamb*()
    np.sum()
    return error 
def als(A, k):
    m, n = A.shape
    # Initialize P, Q
    P = np.ones((k,m))
    Q = np.ones((k,n))
    Id_k = np.eye(N=k)
    # Alternate to optimize
    num_iter = 0
        while num_iter < self.max_iter:
            for u in range(m):
                P[:, u] = np.linalg.inv(Q[]*Q[].T+lamb*Id_k)*np.dot(A[u,:], Q[:, ])
            for i in range(n):
                Q[:, u] = np.linalg.inv(+lamb*Id_k)* 
            if 
                break
            num_iter += 1
     
class ALS(AlgoBase):
    def __init__(self,):
        
    def fit(self, trainset):
        AlgoBase.fit(self, trainset)
        
                
    def estimate(self, u, i):



In [6]:
model = 
model.fit()
predictions = 
rmse = 


TypeError: 'Trainset' object is not subscriptable